In [1]:
import gc
import glob

import geopandas as gpd
import momepy as mm

import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [2]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
simplfied_buildings_dir = '/data/uscuni-ulce/processed_data/simplified_buildings/'


regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [3]:
# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64



# df.region.value_counts().iloc[:20].index.values.astype(int)

# array([     4, 107131,  16242,   4214,   8707,  51289, 114311,  55763,
#         14679,  62954,  10579,  38679,  86873,  34266,     10,  80404,
#         18557,  69333,   1154, 130658])
# '''

##

In [4]:
# 107131 - krakow
# 86873 = vienna
# 69333 = prague
# munich
# 4 - rhineruhr
# 55763 - berlin
#bratislava - 99886
# freiburg - 5883
 # hamburg - 16242
# munich - 38679

In [5]:
region_name = 69333
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [6]:
# region_hulls.explore()

## Buildings


In [30]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [31]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 228 ms, sys: 80.2 ms, total: 308 ms
Wall time: 285 ms


In [32]:
%%time
buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0
Final polygons:  457127 , dropped:  0.02918639964321368
CPU times: user 1min 8s, sys: 57.2 ms, total: 1min 8s
Wall time: 1min 8s


In [33]:
%%time
## the buffer has to be higher than the simplification value
graph = Graph.build_fuzzy_contiguity(buildings, buffer=.25).assign_self_weight()


CPU times: user 8.1 s, sys: 738 μs, total: 8.1 s
Wall time: 8.1 s


In [34]:
connected_building_geoms = buildings.dissolve(graph.component_labels)

In [35]:
cparts = graph.component_labels.value_counts()

In [36]:
connected_building_geoms.loc[cparts.index, 'constituent_parts'] = cparts.values

In [37]:
connected_building_geoms = connected_building_geoms.explode(ignore_index=True)
connected_building_geoms = connected_building_geoms[connected_building_geoms.geom_type == "Polygon"].reset_index(drop=True)

In [38]:
# buildings.to_parquet(simplfied_buildings_dir + f"buildings_{region_id}.parquet")

In [39]:
connected_building_geoms.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

## Streets

In [27]:
from core.generate_streets import process_region_streets, process_single_region_streets

In [28]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:343: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:200: UserWarning: Could not create a connectio

CPU times: user 8min 50s, sys: 2.77 s, total: 8min 53s
Wall time: 8min 47s


In [29]:
# save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [30]:
# streets.explore()

In [31]:
# import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [32]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [7]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [8]:
# 21894
# 47090 - problems with topolgical exceptions

In [9]:
### copy the original buildings
# !cp /data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333_merged.parquet /data/uscuni-ulce/processed_data/buildings/buildings_69333.parquet

In [43]:
%%time
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

---- Processing region:  69333 2024-11-08 19:57:39.832579


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


CPU times: user 1min 28s, sys: 2.08 s, total: 1min 30s
Wall time: 2min 5s


In [44]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

Processed enclosures
processed tesselations


In [12]:
# import lonboard
# layeasr = lonboard.SolidPolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [13]:
# layer = lonboard.SolidPolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [10]:
from core.generate_ngraphs import process_region_graphs

In [46]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 243 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
CPU times: user 39.5 s, sys: 575 ms, total: 40.1 s
Wall time: 40 s


In [ ]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]

In [35]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,
21894,"POLYGON ((4290400 3444300, 4290000 3444400, 42..."
21904,"POLYGON ((4298300 3013900, 4293400 3014600, 42..."
22040,"POLYGON ((4300900 3137100, 4295800 3137600, 42..."
22105,"POLYGON ((4304900 2870800, 4298400 2872300, 42..."
22345,"POLYGON ((4308200 3070700, 4298500 3071500, 42..."
...,...
50528,"POLYGON ((4492200 3191700, 4486500 3193600, 44..."
50880,"POLYGON ((4494600 3230000, 4492100 3230500, 44..."
50896,"POLYGON ((4506400 3167400, 4503300 3168700, 44..."


In [ ]:
# from joblib import Parallel, delayed
# n_jobs = -1
# new = Parallel(n_jobs=n_jobs)(
#     delayed(process_region_graphs)(     region_id,
#                                         graph_dir,
#                                         buildings_dir,
#                                         streets_dir,
#                                         enclosures_dir,
#                                         tessellations_dir
#                                   ) 
#     for region_id, _ in region_hulls.iterrows())

In [38]:
1

1

## Characters

In [11]:
from core.generate_chars import process_single_region_chars

In [48]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-11-08 20:00:27.210400 ----Processing ------ 69333
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:890: Ru

Processing tessellation
CPU times: user 9min 36s, sys: 4.14 s, total: 9min 40s
Wall time: 9min 38s


In [12]:
from core.generate_chars import process_single_region_chars, process_street_chars, process_building_chars, process_tessellation_chars,process_enclosure_chars

In [49]:
# process_building_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [ ]:
# process_street_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [ ]:
# process_tessellation_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [ ]:
# process_enclosure_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [14]:
czech_regions = [ 
    # 55835,  61183,  61187,  63756,  64463,  66593,  67156,  67803, 69333, 
         72507,  73735,  75472,  75642,  83538,  84297,  84962,
        93167,  97978, 100210, 102127, 103882, 105090, 107131]

In [ ]:
for region_id in czech_regions:
    enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)
    enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
    print("Processed enclosures")
    
    ## save files
    tesselations.to_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
    )
    print("processed tesselations")
    process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    )
    process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
    )

---- Processing region:  72507 2024-11-17 13:08:53.379840
Processed enclosures
processed tesselations
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 40 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
2024-11-17 13:09:43.788872 ----Processing ------ 72507
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
---- Processing region:  73735 2024-11-17 13:12:18.957263
Processed enclosures
processed tesselations
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


...

/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
---- Processing region:  83538 2024-11-17 13:19:09.654581
Processed enclosures
processed tesselations
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 50 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
2024-11-17 13:19:56.874106 ----Processing ------ 83538
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


## Generate primary data

In [50]:
from core.generate_merged_primary_chars import merge_into_primary # 47919

In [51]:
%%time
merge_into_primary(region_id)

Processing region 69333
CPU times: user 2.35 s, sys: 838 ms, total: 3.19 s
Wall time: 2.57 s


In [ ]:
1

In [ ]:
1

## Generate morphotopes

In [52]:
from core.generate_clusters import process_single_region_morphotopes

In [53]:
process_single_region_morphotopes(region_id)

2024-11-08 20:10:08.363259 ----Processing ------ 69333
--------Generating lag----------
--------Generating morphotopes----------
--------Generating morphotopes data----------
